In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [2]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

# 3 深度森林算法的简介、实现（不调用sklearn的森林模型）和实验

## 3.1 深度森林的原理
深度森林是一种新的基于决策树的集成学习算法，它通过对树构成的森林进行集成并串联起来达到让分类器做表征学习的目的，从而提高分类的效果。

### 3.1.1 集成学习Bagging简介——从决策树到随机森林和极端随机森林

集成学习算法本身不算一种单独的机器学习算法，而是以牺牲一部分效率为代价，通过构建并结合多个机器学习器来完成学习任务，常见的集成学习方法主要有Bagging和Boosting，下面要讲述的随机森林就是Bagging的代表算法。

Bagging的算法过程为：从原始样本集中使用Bootstraping方法随机从$m$个样本中抽取$m_0$个训练样本，共进行$k$轮抽取，得到$k$个训练集。对于$k$个训练集，我们训练$k$个模型，这$k$个模型不一定是相同的，但它们的重要性相同。整个Bagging过程可以用下图中一个类似并联的形式表示：
![](https://ai-studio-static-online.cdn.bcebos.com/da896d6cca31437ebf6b89bd0808dc9111be4914e3ec43eb9fbc5af7352f01ee)


#### 随机森林算法
对于具体的随机森林算法，在为每棵决策树有放回地选取了一部分训练数据后，决策树的每个节点构建时，还需要从$n$个特征中随机有放回地选取$n_0$个（一般$n_0=\sqrt{n}$），按照基尼指数对决策树进行构建。总而言之，随机森林的随机主要是两个方面，一个是随机有放回的抽取样本，一个是随机选取特征变量。

#### 极端随机森林算法
极端随机森林同样是由多棵决策树集成的分类器，它与随机森林的区别主要如下：

（1）极端随机森林在特征选择时比较激进，在决策树每个结点上只随机选取一个特征来划分决策树，而不像随机森林一样基于信息增益，基尼系数，均方差之类的原则，选择一个最优的特征值划分点；

（2）随机森林为每棵决策树随机选取一定样本，而极端随机森林中，有时候每棵决策树都采用原始训练集。

从（1）可以看出，由于随机选择了特征值的划分点位，而不是最优点位，这样会导致生成的决策树的规模一般会大于随机森林所生成的决策树。也就是说，模型的方差相对于随机森林进一步减少，但是偏差相对于随机森林进一步增大。一般情况下，
极端随机森林分类器在分类精度和训练时间等方面都要优于随机森林分类器。

- 在编写代码时，极端随机森林的实现只需要将普通的随机森林中“每次分裂选取的特征数量”调整为$1$。

### 3.1.2 提出深度森林方法的动机
深度学习方法建立在神经网络模型上，在图像、视频、音频等领域都有着强大的性能，它作为一种多层参数化的可分化非线性模块，可以通过反向传播进行训练。

####然而，深度神经网络DNN也有着明显的缺点：

（1）DNN有着大量需要手动设定的超参数，其学习性能严重依赖于仔细的参数调整，如此之多的干扰因素也使得DNN的理论分析也变得非常困难。

（2）DNN需要大量的数据进行训练，因此DNN很难被应用于只有小规模训练数据的任务中。然而现实情况是，由于高昂的标注成本，大多实际应用场景对应的问题还是小型数据集。

（3）DNN的网络结构必须在训练之前确定，这意味着模型的复杂性无法根据数据输入的情况进行自适应的调整，容易产生过度复杂的神经网络结构。
实际上，在许多Kaggle竞赛中，随机森林和XGBoost算法的效果要显著优于DNN。

####相比之下，深度森林算法：

（1）需要设定的超参数少。

深度森林算法中只有森林中树的类型，树的棵数等少量几个参数需要设置，具体的参数列表和DNN对比如下图：
![](https://ai-studio-static-online.cdn.bcebos.com/a1ad656b90aa41f493d264766ce115aeb226cec6be0741e49389d70bf9626081)


（2）效率高，支持小规模训练数据。

（3）深度森林的结构（也就是级联森林部分的深度）可以通过对数据的训练过程自适应地进行调整，不会产生过度复杂的算法结构。

### 3.1.3 深度森林的两个组件——MGS和gcForest

深度森林主要分为多粒度扫描和级联森林两部分。

#### （1）多粒度扫描(Multi-Grain-Scanning)

在常见的一些问题中，数据本身的不同特征之间，有着显著的时空间关系。例如，在图像识别问题中，位置靠近的像素点之间有很强的空间关系；一组呈现序列形式的数据也有着前后顺序上的关系。为此提出了多粒度扫描，它利用多种大小的滑动窗口进行采样，以获得不同尺度的特征子样本，这一步的处理类似于卷积的过程。

多粒度扫描的一个例子如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/7d1cb52c035e41b3bb4ce6602ca50c373f1e008784e24aad8e5a22a0a307b879)


对整个MGS的过程进行举例分析如下：

以一维情况的三分类问题为例，对于一个$400$维的输入特征，如果选用一个$100$维的窗口，窗口滑动的步长为$1$，则会产生$400-100+1=301$个大小为$100$维的向量。这$301$个向量在经过随机森林$A$和一个极端随机森林$B$后（每个森林输出一个分类概率的$3$维向量），把这些三维向量全部拼到一起，构成一个$301\times 3\times 2=1806$维的向量。至此MGS全部完成。

- 需要注意的是，在编写MGS的程序时，首先要使用全部的训练数据对两个森林进行训练，训练完毕后，再将所有的特征向量输入MGS，把得到的向量再附上原有的标签，即可完成MGS过程。


#### （2）级联森林(Cascade-Forest)

级联森林的每一层都包含许多个集成学习分类器，它的本质是集成的集成。为了使生成的特征具有多样性，在每一层中采用随机森林和极端随机森林这两种不同的森林结构，这两种森林已经在3.1.1中介绍。需要强调的是，这两种森林中决策树的每个叶子结点最多能容纳一个样本。

如下图，假设所有标签构成的集合的大小为$C$，在级联森林的具体结构中，输入的特征向量经过每个森林都会生成一个$C$维向量，假设每层有$F$个森林，那么每一层输出的特征向量就是$FC$维的。之后，级联森林采取了类似DenseNet的结构，将这个向量直接与原来的特征向量合并（而不是像ResNet一样相加），作为下一层的输入，直到最后一层，输出$F$个$C$维向量，把它们取平均即可得到最后的$C$维预测结果。

![](https://ai-studio-static-online.cdn.bcebos.com/62d29b0a30e74dbb9778fb71e52279de737b5e70d1d3438cad7cdb9088660108)

	
需要注意的是，为了降低过拟合风险，每个森林生成的类向量是通过K折交叉验证产生的，也就是说，每个样本都被当作$K-1$次训练数据，产生$k-1$个$C$维向量，然后对其取平均值，生成图中的$C$维向量。

每个森林具体的决策过程如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/897f849b825f4cee96edc97c4f0ef752603a4826837f452eb173c9fd8bff3208)

### 3.1.4 深度森林的整体结构

如下图，假设有$w$种不同大小的窗口，那么在$N$层级联森林中，每层都分别将该层中的森林输出的特征和不同大小窗口产生的输入特征向量做拼接，相当于级联森林实际上是有$w\times N$层的。MGS和gcForest的运作均根据3.1.3中所述的流程进行。

![](https://ai-studio-static-online.cdn.bcebos.com/39876bf9c0aa431ca17f1fc6d31c7776fdbcb249d07148bd85ffe9ef8eced6b4)

以上就是深度森林的整个算法流程。

## 3.2 用Python+Paddle实现随机森林

这里需要特别强调的是，在我们实现深度森林的过程中，没有调用
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraForestClassifier
```
这两种sklearn中提供的森林结构，所有算法的部分都是纯Python代码实现的。深度森林的两个组件以及原生Python实现的随机森林都封装在类中。

首先是随机森林和极端随机森林的代码实现：

(其中需要注意的是，深度森林算法中的每个森林结构都需要输出概率值，因此需要重新编写计算概率的函数)

In [9]:
import pandas as pd
import numpy as np
import paddle
import random
import math
import collections
from joblib import Parallel, delayed

In [10]:
# 纯Python+Paddle实现随机森林

def calculate_prob(arr,Y_label):  # 计算概率的函数
    # 给出[5,4,5,5,5,6]，要返回[1/3,1/2,1/6]
    prob=[]
    dic=dict()
    for item in arr:
        if str(item) in dic:
            dic[str(item)]+=1
        else:
            dic[str(item)]=1
    skeys=sorted(dic.keys())
    for item in Y_label:
        if str(item) in dic:
            prob.append(dic[str(item)]/len(arr))
        else:
            prob.append(0)
    return prob


class Tree(object):
    """定义一棵决策树"""

    def __init__(self):
        self.split_feature = None
        self.split_value = None
        self.leaf_value = None
        self.tree_left = None
        self.tree_right = None

    def calc_predict_value(self, dataset):
        """通过递归决策树找到样本所属叶子节点"""
        if self.leaf_value is not None:
            return self.leaf_value
        elif dataset[self.split_feature] <= self.split_value:
            return self.tree_left.calc_predict_value(dataset)
        else:
            return self.tree_right.calc_predict_value(dataset)


class RandomForestClassifier(object):
    def __init__(self, n_estimators=10, max_depth=-1, min_samples_split=2, min_samples_leaf=1,
                 min_split_gain=0.0, colsample_bytree=None, subsample=0.8, random_state=2021):
        """
        随机森林参数
        ----------
        n_estimators:      树数量
        max_depth:         树深度，-1表示不限制深度
        min_samples_split: 节点分裂所需的最小样本数量，小于该值节点终止分裂
        min_samples_leaf:  叶子节点最少样本数量，小于该值叶子被合并
        min_split_gain:    分裂所需的最小增益，小于该值节点终止分裂
        colsample_bytree:  列采样设置，可取[sqrt、log2]。sqrt表示随机选择sqrt(n_features)个特征，
                           log2表示随机选择log(n_features)个特征，设置为其他则不进行列采样
        subsample:         行采样比例
        random_state:      随机种子，设置之后每次生成的n_estimators个样本集不会变，确保实验可重复
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth if max_depth != -1 else float('inf')
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_split_gain = min_split_gain
        self.colsample_bytree = colsample_bytree
        self.subsample = subsample
        self.random_state = random_state
        self.trees = None
        self.feature_importances_ = dict()

    def fit(self, dataset, targets):
        """模型训练入口"""
        targets = targets.to_frame(name='label')

        if self.random_state:
            random.seed(self.random_state)
        random_state_stages = random.sample(range(self.n_estimators), self.n_estimators)

        # 两种列采样方式
        if self.colsample_bytree == "sqrt":
            self.colsample_bytree = int(len(dataset.columns) ** 0.5)
        elif self.colsample_bytree == "log2":
            self.colsample_bytree = int(math.log(len(dataset.columns)))
        elif self.colsample_bytree == 1:
            self.colsample_bytree = 1
        else:
            self.colsample_bytree = len(dataset.columns)

        # 并行建立多棵决策树
        self.trees = Parallel(n_jobs=-1, verbose=0, backend="threading")(
            delayed(self._parallel_build_trees)(dataset, targets, random_state)
            for random_state in random_state_stages)

    def _parallel_build_trees(self, dataset, targets, random_state):
        """bootstrap有放回抽样生成训练样本集，建立决策树"""
        subcol_index = random.sample(dataset.columns.tolist(), self.colsample_bytree)
        dataset_stage = dataset.sample(n=int(self.subsample * len(dataset)), replace=True,
                                       random_state=random_state).reset_index(drop=True)
        dataset_stage = dataset_stage.loc[:, subcol_index]
        targets_stage = targets.sample(n=int(self.subsample * len(dataset)), replace=True,
                                       random_state=random_state).reset_index(drop=True)

        tree = self._build_single_tree(dataset_stage, targets_stage, depth=0)
        # print(tree.describe_tree())
        return tree

    def _build_single_tree(self, dataset, targets, depth):
        """递归建立决策树"""
        # 如果该节点的类别全都一样/样本小于分裂所需最小样本数量，则选取出现次数最多的类别。终止分裂
        if len(targets['label'].unique()) <= 1 or dataset.__len__() <= self.min_samples_split:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

        if depth < self.max_depth:
            best_split_feature, best_split_value, best_split_gain = self.choose_best_feature(dataset, targets)
            left_dataset, right_dataset, left_targets, right_targets = \
                self.split_dataset(dataset, targets, best_split_feature, best_split_value)

            tree = Tree()
            # 如果父节点分裂后，左叶子节点/右叶子节点样本小于设置的叶子节点最小样本数量，则该父节点终止分裂
            if left_dataset.__len__() <= self.min_samples_leaf or \
                    right_dataset.__len__() <= self.min_samples_leaf or \
                    best_split_gain <= self.min_split_gain:
                tree.leaf_value = self.calc_leaf_value(targets['label'])
                return tree
            else:
                # 如果分裂的时候用到该特征，则该特征的importance加1
                self.feature_importances_[best_split_feature] = \
                    self.feature_importances_.get(best_split_feature, 0) + 1

                tree.split_feature = best_split_feature
                tree.split_value = best_split_value
                tree.tree_left = self._build_single_tree(left_dataset, left_targets, depth + 1)
                tree.tree_right = self._build_single_tree(right_dataset, right_targets, depth + 1)
                return tree
        # 如果树的深度超过预设值，则终止分裂
        else:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

    def choose_best_feature(self, dataset, targets):
        """寻找最好的数据集划分方式，找到最优分裂特征、分裂阈值、分裂增益"""
        best_split_gain = 1
        best_split_feature = None
        best_split_value = None

        for feature in dataset.columns:
            if dataset[feature].unique().__len__() <= 100:
                unique_values = sorted(dataset[feature].unique().tolist())
            # 如果该维度特征取值太多，则选择100个百分位值作为待选分裂阈值
            else:
                unique_values = np.unique([np.percentile(dataset[feature], x)
                                           for x in np.linspace(0, 100, 100)])

            # 对可能的分裂阈值求分裂增益，选取增益最大的阈值
            for split_value in unique_values:
                left_targets = targets[dataset[feature] <= split_value]
                right_targets = targets[dataset[feature] > split_value]
                split_gain = self.calc_gini(left_targets['label'], right_targets['label'])

                if split_gain < best_split_gain:
                    best_split_feature = feature
                    best_split_value = split_value
                    best_split_gain = split_gain
        return best_split_feature, best_split_value, best_split_gain

    @staticmethod
    def calc_leaf_value(targets):
        """选择样本中出现次数最多的类别作为叶子节点取值"""
        label_counts = collections.Counter(targets)
        major_label = max(zip(label_counts.values(), label_counts.keys()))
        return major_label[1]

    @staticmethod
    def calc_gini(left_targets, right_targets):
        """分类树采用基尼指数作为指标来选择最优分裂点"""
        split_gain = 0
        for targets in [left_targets, right_targets]:
            gini = 1
            # 统计每个类别有多少样本，然后计算gini
            label_counts = collections.Counter(targets)
            for key in label_counts:
                prob = label_counts[key] * 1.0 / len(targets)
                gini -= prob ** 2
            split_gain += len(targets) * 1.0 / (len(left_targets) + len(right_targets)) * gini
        return split_gain

    @staticmethod
    def split_dataset(dataset, targets, split_feature, split_value):
        """根据特征和阈值将样本划分成左右两份，左边小于等于阈值，右边大于阈值"""
        left_dataset = dataset[dataset[split_feature] <= split_value]
        left_targets = targets[dataset[split_feature] <= split_value]
        right_dataset = dataset[dataset[split_feature] > split_value]
        right_targets = targets[dataset[split_feature] > split_value]
        return left_dataset, right_dataset, left_targets, right_targets

    def predict(self, dataset):
        """输入样本，预测所属类别"""
        res = []
        dataset = pd.DataFrame(dataset)
        for _, row in dataset.iterrows():
            pred_list = []
            # 统计每棵树的预测结果，选取出现次数最多的结果作为最终类别
            for tree in self.trees:
                pred_list.append(tree.calc_predict_value(row))
            pred_label_counts = collections.Counter(pred_list)
            pred_label = max(zip(pred_label_counts.values(), pred_label_counts.keys()))
            res.append(pred_label[1])
        return np.array(res)

    def predict_prob(self, dataset, Y_label):
        res = []
        # Y转成整数形式
        Y=np.array(Y_label)
        Y=Y.astype(int)
        Y_label=Y.tolist()
        for _, row in dataset.iterrows():
            pred_list = []
            # 统计每棵树的预测结果，选取出现次数最多的结果作为最终类别
            for tree in self.trees:
                pred_list.append(tree.calc_predict_value(row))
            pred_list=np.array(pred_list)
            pred_list=pred_list.astype(int)
            pred_list=pred_list.tolist()
            pred_prob=calculate_prob(pred_list,Y_label)
            res.append(pred_prob)
        return np.array(res)

之后是MGS结构的代码实现，其中包括MGS入口，区分训练MGS或执行MGS，窗口滑动的实现这三个函数：

In [11]:
class MGS(object):
    '''
    输入参数说明：
    windows：滑动窗口的大小构成的数组
    n_mgsTree：mgs的每个森林中树的棵数
    min_sample_mgs：mgs每个决策树叶子结点最多能容纳的样本数
    stride：滑动窗口的步长
    另外，由于个人电脑内存原因，mgs中的森林数设置为2，不再设置为其余2的倍数。
    '''
    def __init__(self,windows,n_mgsTree,min_sample_mgs,stride):
        self.windows=windows
        self.n_mgsTree=n_mgsTree
        self.min_sample_mgs=min_sample_mgs
        self.stride=stride
        self.prfs=[]
        self.crfs=[]

    def mg_scanning(self,X, y=None):  # MGS入口函数
        windows=self.windows
        self._n_samples=np.shape(X)[0] # 样本个数
        shape_1X = len(X[0]) # 样本特征维度
        if isinstance(shape_1X, int):
            shape_1X = [1, shape_1X]  # 把一维特征拼成list
        mgs_pred_prob = []

        for i in range(len(self.windows)):
            if y is None:  # 测试
                #print(self.prfs)
                wdw_pred_prob = self.window_slicing_pred_prob(X, self.windows[i], shape_1X, y=y,prff=self.prfs[i],crff=self.crfs[i])
                mgs_pred_prob.append(wdw_pred_prob)
            else:  # 训练
                prf,crf=self.window_slicing_pred_prob(X, self.windows[i], shape_1X, y=y,prff=None,crff=None)
                self.prfs.append(prf)
                self.crfs.append(crf)
        if len(mgs_pred_prob)!=0:
            #(mgs_pred_prob)
            return np.concatenate(mgs_pred_prob, axis=1)
        else:
            return

    def window_slicing_pred_prob(self, X, window, shape_1X, y ,prff,crff):  # 区分是训练MGS还是执行MGS
        n_tree = self.n_mgsTree
        min_samples = self.min_sample_mgs
        stride = self.stride
        sliced_X, sliced_y = self._window_slicing_sequence(X, window, shape_1X, y=y, stride=stride)
        if y is not None:
            print('MGS Training...')
            prf = RandomForestClassifier(n_estimators=n_tree, colsample_bytree='sqrt',
                                         min_samples_split=min_samples)
            crf = RandomForestClassifier(n_estimators=n_tree, colsample_bytree=1,
                                         min_samples_split=min_samples)
            sliced_X=pd.DataFrame(sliced_X)
            sliced_y=pd.Series(sliced_y)
            prf.fit(sliced_X, sliced_y)
            crf.fit(sliced_X, sliced_y)
            self.all_label=list(set(sliced_y))
            return prf,crf

        else:
            print('MGS Executing...')
            sliced_X = pd.DataFrame(sliced_X)
            pred_prob_prf = prff.predict_prob(sliced_X,self.all_label)
            #print("*"*100)
            #print(pred_prob_prf)
            pred_prob_crf = crff.predict_prob(sliced_X,self.all_label)
            pred_prob = np.c_[pred_prob_prf, pred_prob_crf]
            return pred_prob.reshape([self._n_samples, -1])

    def _window_slicing_sequence(self, X, window, shape_1X, y=None, stride=1):  # 真正的窗口滑动函数
        if shape_1X[1] < window:
            raise ValueError('window must be smaller than the sequence dimension')

        len_iter = np.floor_divide((shape_1X[1] - window), stride) + 1
        iter_array = np.arange(0, stride*len_iter, stride)

        ind_1X = np.arange(np.prod(shape_1X))
        inds_to_take = [ind_1X[i:i+window] for i in iter_array]
        sliced_sqce = np.take(X, inds_to_take, axis=1).reshape(-1, window)

        if y is not None:
            sliced_target = np.repeat(y, len_iter)
        else:
            sliced_target = None

        return sliced_sqce, sliced_target

下面对iris数据集执行一次MGS过程，作为示例（由于iris本身的特征很少，窗口大小直接设为[1,3]）：

In [12]:
from sklearn.model_selection import train_test_split
# 这里并没有使用sklearn中的模型

MJ=MGS(windows=[1,3],n_mgsTree=2,min_sample_mgs=1,stride=1)
df = pd.read_csv("iris.txt", sep='\t', header=None)  # 注意导入的数据集有没有列名， 没有的话就加入 header=None ，否则不加
df = np.array(df)
x = df[:, :-1]
y = df[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10086)
MJ.mg_scanning(x_train,y_train)
Data=MJ.mg_scanning(x_train)
print("MGS之前数据维度：",x_train.shape)
print("MGS之后数据维度：",Data.shape)

MGS Training...
MGS Training...
MGS Executing...
MGS Executing...
MGS之前数据维度： (105, 4)
MGS之后数据维度： (105, 36)


之后是级联森林部分的实现，分为级联森林的构建和级联森林的训练两个类实现：

In [13]:
# 定义 deepforest 级联层类
class CascadeLayer(object):
    '''
    参数说明：
    num_forsts：森林个数
    n_estimators：每个森林中树木棵数
    max_depth：每棵树的最大深度
    min_samples_leaf：叶子结点最小样本数
    '''
    def __init__(self, n_estimators, num_forests, max_depth=40, min_samples_leaf=1):  
        self.min_samples_leaf = min_samples_leaf  
        self.model = []  # 结果类向量
        self.num_forests = num_forests
        self.n_estimators = n_estimators
        self.max_depth = max_depth

    def fit(self, X, y, y_col):  # 训练函数
        proba = np.zeros([self.num_forests, y_col.shape[0]])    
        for index in range(self.num_forests):  
            if index % 2 == 0:  # 偶数个森林是随机森林
                clf = RandomForestClassifier(n_estimators=5,
                                             max_depth=5,
                                             min_samples_split=6,
                                             min_samples_leaf=2,
                                             min_split_gain=0.0,
                                             colsample_bytree="sqrt",
                                             subsample=0.8,
                                             random_state=66)  # 这里调用原生的随机森林（sklearn中没有colsample_bytree这一属性）
                train_data, train_label = pd.DataFrame(X), pd.Series(y)
                clf.fit(train_data, train_label)
                y_pd = pd.DataFrame(y_col)
                proba[index, :] = clf.predict(y_pd)  
            else:  # 奇数个森林是极端森林
                clf = RandomForestClassifier(n_estimators=5,
                                             max_depth=5,
                                             min_samples_split=6,
                                             min_samples_leaf=2,
                                             min_split_gain=0.0,
                                             colsample_bytree=1,  # 极端森林每次只选取一个特征，而不是sqrt
                                             subsample=0.8,
                                             random_state=66)  # 这里调用原生的极端随机森林
                train_data, train_label = pd.DataFrame(X), pd.Series(y)
                clf.fit(train_data, train_label)
                y_pd = pd.DataFrame(y_col)
                proba[index, :] = clf.predict(y_pd)  
            self.model.append(clf)  # 创建新的级联森林层

        average_proba = np.sum(proba, axis=0)  
        average_proba /= self.num_forests  
        val_concatenate = proba.transpose((1, 0))  
        return average_proba, val_concatenate  # 返回平均结果和转置后的类向量矩阵

    def predict(self, test_data):  # 定义预测函数
        predict_prob = np.zeros([self.num_forests, test_data.shape[0]])
        for forest_index, clf in enumerate(self.model):
            predict_prob[forest_index, :] = clf.predict(test_data)

        predict_avg = np.sum(predict_prob, axis=0)
        predict_avg /= self.num_forests
        predict_concatenate = predict_prob.transpose((1, 0))
        return predict_avg, predict_concatenate


# K折交叉检验训练级联森林
class CascadeTrain(object):
    '''
    参数：
    num_forests：每层的森林数量
    n_estimators：森林中树的数量
    n_fold：k折的k
    kf：交叉验证的下标
    layer_index：层索引
    max_depth：树的最大深度
    min_samples_leaf：叶子结点最小样本数
    '''
    def __init__(self, num_forests, n_estimators, n_fold, kf, layer_index, max_depth=31, min_samples_leaf=1):
        self.num_forests = num_forests
        self.n_estimators = n_estimators
        self.n_fold = n_fold
        self.kf = kf
        self.layer_index = layer_index
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.model = []

    def fit(self, X, y):
        num_samples, num_features = X.shape
        proba = np.empty([num_samples])
        matrix1 = np.empty([num_samples, self.num_forests])
        for train_index, test_index in self.kf:  
            X_train = X[train_index, :]
            X_val = X[test_index, :]
            y_train = y[train_index]
            gclayer = CascadeLayer(self.n_estimators, self.num_forests, self.max_depth, self.min_samples_leaf)
            proba[test_index], matrix1[test_index, :] = gclayer.fit(X_train, y_train, X_val)
            self.model.append(gclayer)  
        return [proba, matrix1]

    def predict(self, X):  # 定义预测函数，用做下一层的训练数据
        pred_prob = np.zeros([X.shape[0]])
        for layer in self.model:
            temp_prob, temp_prob_concatenate = layer.predict(X)
            pred_prob += temp_prob
        pred_prob /= self.n_fold
        return pred_prob

最后是整个深度森林对应的类：

In [14]:
# 定义deepforest类
class gcForest(object):
    def __init__(self, num_estimator, num_forests, max_layer=100, max_depth=30, n_fold=6):
        # 分别是森林个数，森林中树的个数，森林最大层数，最大深度，交叉验证折数
        self.num_estimator = num_estimator
        self.num_forests = num_forests
        self.n_fold = n_fold
        self.max_depth = max_depth
        self.max_layer = max_layer
        self.model = []     # 保存森林中的model

    def fit(self, X, y):
        train_loss = 0.0
        layer = 0
        top_layer = 0
        judge_layer = 0

        kf_flod = KFold(self.n_fold).split(X)
        print('================================')
        print('Model starts...')
        print('--------------------------------')
        # 开始验证
        while layer < self.max_layer:
            # 初始化森林新的layer
            new_layer = CascadeTrain(self.num_forests, self.num_estimator, self.n_fold,
                                     kf_flod, layer, self.max_depth, 1)
            # 当前层进行训练
            proba, gcf_stack = new_layer.fit(X, y)
            # 计算当前层的预测loss
            temp_val_loss = self.eval(y, proba)
            print("layer : {} | Val Loss : {:.6f}".format(layer, temp_val_loss))
            # loss下降小于阈值，停止增长
            if train_loss < temp_val_loss:
                judge_layer += 1
            else:
                judge_layer = 0
                train_loss = temp_val_loss
                top_layer = layer
            if judge_layer >= 3:
                break
            # 开始下一层
            layer = layer + 1
            self.model.append(new_layer)
        print('================================')

        for index in range(len(self.model), top_layer + 1, -1):  # 删除多余的层
            self.model.pop()

    def predict(self, X):
        for layer in self.model:
            prediction = layer.predict(X)
        return prediction

    def eval(self, real, predict):  # 计算预测准确率
        temp = np.log(abs(real + 1)) - np.log(abs(predict + 1))
        res = np.dot(temp, temp) / len(temp)
        return res

最后是整个深度森林模型的测试：（注意，全程没有调用sklearn.ensemble）

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from time import time

def main():
    df = pd.read_csv("iris.txt", sep='\t', header=None)  # 注意导入的数据集有没有列名， 没有的话就加入 header=None ，否则不加
    df = np.array(df)
    x = df[:, :-1]
    y = df[:, -1]
    # MGS训练+执行
    start0=time()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2021)
    MJ = MGS(windows=[1, 3], n_mgsTree=10, min_sample_mgs=1, stride=1)
    MJ.mg_scanning(x_train, y_train)
    Data_train = MJ.mg_scanning(x_train)
    MJ.mg_scanning(x_test, y_test)
    Data_test = MJ.mg_scanning(x_test)
    end0=time()
    print('MGS执行时间：',end0-start0)
    print(x_train.shape)
    print(Data_train.shape)
    print(Data_test.shape)
    clf = gcForest(num_estimator=100, num_forests=4, max_layer=2, max_depth=100, n_fold=5)
    # 训练
    start = time()
    clf.fit(Data_train, y_train)
    end = time()
    print("\ntraining time: {:.3f} s".format(end - start))
    # 预测
    start = time()
    pred_y = clf.predict(Data_test).astype('int')  # 预测标签转应为整数类型
    end = time()
    print("prediction time: {:.3f} s".format(end - start))
    acc = accuracy_score(y_test, pred_y) * 100
    print("\n带MGS的深度森林在iris上的准确率: {:.3f} %".format(acc))

    clf = gcForest(num_estimator=100, num_forests=4, max_layer=2, max_depth=100, n_fold=5)
    start = time()
    clf.fit(x_train, y_train)
    end = time()
    print("\ntraining time: {:.3f} s".format(end - start))
    # 预测
    start = time()
    pred_y = clf.predict(x_test).astype('int')  # 预测标签转应为整数类型
    end = time()
    print("prediction time: {:.3f} s".format(end - start))
    acc = accuracy_score(y_test, pred_y) * 100
    print("\n不带MGS的深度森林在iris上的准确率: {:.3f} %".format(acc))

main()

MGS Training...
MGS Training...
MGS Executing...
MGS Executing...
MGS Training...
MGS Training...
MGS Executing...
MGS Executing...
MGS执行时间： 31.151531219482422
(105, 4)
(105, 36)
(45, 36)
Model starts...
--------------------------------
layer : 0 | Val Loss : 0.057244
layer : 1 | Val Loss : 0.752585

training time: 11.512 s
prediction time: 0.088 s

带MGS的深度森林在iris上的准确率: 91.111 %
Model starts...
--------------------------------
layer : 0 | Val Loss : 0.006121
layer : 1 | Val Loss : 0.621897

training time: 10.631 s
prediction time: 0.079 s

不带MGS的深度森林在iris上的准确率: 93.333 %


值得注意的是，MGS层在数据集的特征很少（如iris的4个特征）时，有可能会起到一定的消极作用，因为窗口可能会变得过小，甚至发生1个特征输入MGS的两种森林的情况。由此可见，在数据集很小时，可以考虑不使用MGS层，只使用后面的gcForest层。

## 3.3 深度森林算法在百余个测试数据集上的结果展示

在文件final_result3.3中，我们给出了在160个测试数据集上的结果。这些结果是实现了问题四中的几种交叉检验方法（这几种方法的实现参见下文）后一起运行深度森林和随机森林方法得到的。


其中每一列的意义如下所示：（这个结果同样用于第四问）

- CV_Procedure是选用的交叉检验方法；

- n_rows是数据集的行数，也就是样本个数；

- n_cols是数据集的列数，也就是特征个数；

- EPE_DF是K折交叉检验后深度森林方法的准确率的均值；

- VAR_DF是K折交叉检验后深度森林方法的准确率的方差；

- Time_DF是深度森林方法运行用时；

- EPE_RF是K折交叉检验后随机森林方法的准确率的均值；

- VAR_RF是K折交叉检验后随机森林方法的准确率的方差；

- Time_RF是随机森林方法运行用时。

深度森林方法、随机森林方法的准确率对比请重点关注EPE_DF和EPE_RF两列。

# 4 探究对比深度森林与随机森林
本检验方案旨在通过不同的交叉验证算法，尽可能消除抽样对算法性能的影响，分别使用**BDS交叉验证算法**、**蒙特卡洛交叉验证算法**、**分层交叉验证算法**、**留一交叉验证算法**测试深度森林算法和随机森林算法的准确率、稳定性与速度，从而得到哪种算法更为优异。
## 4.1 BDS-Cross-Validation

我们使用BDS交叉验证算法，测试算法准确率、稳定性与速度

我们这里直接给出伪代码：

![](https://ai-studio-static-online.cdn.bcebos.com/cedf56ed0c924782a46889b8710f59cbcf2c522036a64694971eb7a904709d7c)

大家如果对该交叉验证具体数学证明感兴趣，可以读山东大学数据科学研究院副院长郭亮教授的paper：

[Subsampling Bias and The Best-Discrepancy Systematic Cross Validation](http://arxiv.org/abs/1907.02437)



In [1]:
# 从.txt文件中读取数据
def read_data(path):
    dataset = []
    with open(path) as f:
        for row in f:
            if "\t\t"in row:
                return np.array(dataset)
            row = row.replace("\t\n","")
            row = row.replace("\n","")
            rows = row.split("\t")
            if len(rows)==1:
                continue
            dataset.append(np.array([float(i) for i in rows]))
    return np.array(dataset)

In [ ]:
import numpy as np
import time


# 生成BDS序列
def generate_BDS(n):
    temp = np.array([np.array([i*np.e - int(i*np.e)]) for i in range(1, n + 1)])
    rank = np.array([np.array([i]) for i in range(n)])
    BDS = np.hstack((temp, rank))
    return BDS[np.argsort(BDS[:, 0]), -1].astype(np.int)


# 选择降维方法，默认使用PCA方法进行降维
def reduce_dimension(Data, method="PCA"):
    from sklearn.decomposition import PCA, FactorAnalysis, KernelPCA, TruncatedSVD
    if method == 'PCA':
        reduce_model = PCA(n_components=1)
    elif method == 'FactorAnalysis':
        reduce_model = FactorAnalysis(n_components=1)
    elif method == 'KernelPCA':
        reduce_model = KernelPCA(kernel='rbf', n_components=1)
    elif method == 'TruncatedSVD':
        reduce_model = TruncatedSVD(1)
    return reduce_model.fit_transform(Data)


# 单个降维方法进行BDSCV操作
def BDSCV_sub(D, reduce_method,model,k):
    # Todo: 通过降维算法把D降至1维
    E = reduce_dimension(D, reduce_method)
    # Todo: 把E添加到D的最后一列
    new_D = np.hstack((D,E))
    # Todo: 根据最后一列升序排序，删除最后一列
    sort_D = new_D[np.argsort(new_D[:, -1]), :-1]+1
    R = generate_BDS(len(sort_D))
    from sklearn.model_selection import KFold
    k_Cross = KFold(n_splits=k)
    score_list = np.array([])
    for train, test in k_Cross.split(R):
        train_index, test_index = R[train], R[test]
        train_data, train_label = sort_D[train_index, :-1], sort_D[train_index, -1]
        test_data, test_label = sort_D[test_index, :-1], sort_D[test_index, -1]
        if model=="df":
            model = CascadeForestClassifier(n_trees=500,verbose=0,random_state=1)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        else:
            model = RandomForestClassifier(n_estimators=500)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        from sklearn.metrics import accuracy_score
        score = accuracy_score(pred, test_label)
        score_list = np.append(score_list, score)
    return score_list


# BDSCV测试模型的性能
def BDS_Cross_Validation(D, model, k):
    start_start = time.time()
    EPEs = []
    VARs = []
    score_pca = BDSCV_sub(D, "PCA", model, k)
    score_fa = BDSCV_sub(D, "FactorAnalysis", model,k)
    score_kpca = BDSCV_sub(D, 'KernelPCA', model,k)
    score_tsvd = BDSCV_sub(D, 'TruncatedSVD', model,k)
    EPEs.append(np.mean(score_pca))
    EPEs.append(np.mean(score_fa))
    EPEs.append(np.mean(score_kpca))
    EPEs.append(np.mean(score_tsvd))
    VARs.append(np.var(score_pca))
    VARs.append(np.var(score_fa))
    VARs.append(np.var(score_kpca))
    VARs.append(np.var(score_tsvd))
    EPEs,VARs = np.array(EPEs),np.array(VARs)
    index = np.argmin(EPEs)
    end_start = time.time()
    return EPEs[index], VARs[index], end_start-start_start

##  4.2 Monte_Carlo_Cross_Validation(MCCV)
蒙特卡洛交叉验证算法，测试算法准确率、稳定性与速度

In [ ]:
import time

# 蒙特卡洛交叉验证（默认重复50遍）
def Monte_Carlo_Cross_Validation(dataset,model,k,repeat=50):
    from sklearn.model_selection import KFold as MCCV
    from sklearn.metrics import accuracy_score
    start_time = time.time()
    EPEs = np.array([])
    VARs = np.array([])
    for i in range(repeat):
        k_Cross = MCCV(n_splits=k, random_state=1, shuffle=True)
        score_list = np.array([])
        data,label = dataset[:,:-1],dataset[:,-1]
        for train_index, test_index in k_Cross.split(dataset):
            train_data, train_label = dataset[train_index, :-1], dataset[train_index, -1]
            test_data, test_label = dataset[test_index, :-1], dataset[test_index, -1]
            if model=="df":
                model = CascadeForestClassifier(n_trees=500,verbose=0,random_state=1)
                model.fit(train_data, train_label)
                pred = model.predict(test_data)
            else:
                model = RandomForestClassifier(n_estimators=500)
                model.fit(train_data, train_label)
                pred = model.predict(test_data)
            score = accuracy_score(pred, test_label)
            score_list = np.append(score_list, score)
        EPEs = np.append(EPEs,np.mean(score_list))
        VARs = np.append(VARs,np.var(score_list))
    end_time = time.time()
    return np.mean(EPEs),np.mean(VARs),end_time-start_time

## 4.3 Stratified_MCCV
分层交叉验证算法，测试算法准确率、稳定性与速度

In [ ]:
# 分层交叉验证
def Monte_Carlo_Cross_Validation_Stratified(dataset,model,k):
    from sklearn.model_selection import StratifiedKFold as Stratified_MCCV
    from sklearn.metrics import accuracy_score
    start_time = time.time()
    k_Cross = Stratified_MCCV(n_splits=k, random_state=1, shuffle=True)
    score_list = np.array([])
    data,label = dataset[:,:-1],dataset[:,-1]
    for train_index, test_index in k_Cross.split(data,label):
        train_data, train_label = data[train_index, :], label[train_index]
        test_data, test_label = data[test_index, :], label[test_index]
        if model=="df":
            model = CascadeForestClassifier(n_trees=500,verbose=0,random_state=1)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        else:
            model = RandomForestClassifier(n_estimators=500)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        score = accuracy_score(pred, test_label)
        score_list = np.append(score_list, score)
    end_time = time.time()
    return np.mean(score_list),np.var(score_list),end_time-start_time


## 4.4 LeaveOneOut(LOO)
留一交叉验证算法，测试算法准确率、稳定性与速度

In [ ]:
# 留一交叉验证
def LeaveOneOut(dataset,model):
    from sklearn.model_selection import LeaveOneOut as LOO
    from sklearn.metrics import accuracy_score
    start_time = time.time()
    loo = LOO()
    score_list = np.array([])
    for train_index, test_index in loo.split(dataset):
        train_data, train_label = dataset[train_index, :-1], dataset[train_index, -1]
        test_data, test_label = dataset[test_index, :-1], dataset[test_index, -1]
        if model=="df":
            model = CascadeForestClassifier(n_trees=500,verbose=0,random_state=1)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        else:
            model = RandomForestClassifier(n_estimators=500)
            model.fit(train_data, train_label)
            pred = model.predict(test_data)
        score = accuracy_score(pred, test_label)
        score_list = np.append(score_list, score)
    end_time = time.time()
    return np.mean(score_list),np.var(score_list),end_time-start_time

## 4.5 针对单个算法测试单个txt范例
分别使用**BDS交叉验证算法**、**蒙特卡洛交叉验证算法**、**分层交叉验证算法**、**留一交叉验证算法**测试随机森林算法

In [ ]:
def result_out(m,v,t):
    print("Mean:\t",m)
    print("Var:\t",v)
    print("Time:\t",t)

D = read_data("randomforest_datasets/analcatdata_asbestos.txt")
m,v,t = BDS_Cross_Validation(D,'rf', 5)
print("---------------------BDS_Cross_Validation---------------------")
result_out(m,v,t)

m,v,t = Monte_Carlo_Cross_Validation(D,'rf',5,repeat=50)
print("---------------------Monte_Carlo_Cross_Validation---------------------")
result_out(m,v,t)

m,v,t = Monte_Carlo_Cross_Validation_Stratified(D,'rf',5)
print("---------------------Monte_Carlo_Cross_Validation_Stratified---------------------")
result_out(m,v,t)

m,v,t = LeaveOneOut(D,'rf')
print("---------------------LeaveOneOut---------------------")
result_out(m,v,t)

---------------------BDS_Cross_Validation---------------------
EPE:	0.7830882352941176
VAR:	0.009515570934256052
Time :	10.008927822113037s

---------------------Monte_Carlo_Cross_Validation---------------------
EPE:	0.785294117647059
VAR:	0.008772707612456744
Time :	114.31486201286316s

---------------------Monte_Carlo_Cross_Validation_Stratified---------------------
EPE:	0.7720588235294118
VAR:	0.003114186851211072
Time :	2.2797107696533203s

---------------------LeaveOneOut---------------------
EPE:	0.7710843373493976
VAR:	0.17651328204383798
Time :	38.19196939468384s



## 4.6 测试对比深度森林和随机森林的表现

以[160个公开数据集](http://aistudio.baidu.com/aistudio/datasetdetail/89765)为例，展示如何对比深度森林和随机森林算法性能。鉴于测试所有数据所需时间过长，这里给出测试5个数据集

"auto.txt","analcatdata_asbestos.txt","analcatdata_boxing1.txt","iris.txt","postoperative-patient-data.txt"

的测试结果，以及测试所有数据集的final_result.xlsx

In [ ]:
import os

# 将结果写入文件file_path
def write_result(data,file_path):
    with open(file_path,"a") as f:
        for e in data:
            f.write(str(e))
            f.write(",")
        f.write("\n")

# 获取所有txt文件
def get_file_path(dir):
    file_dir_list = []
    for root, dirs, files in os.walk(dir, topdown=False):
        for name in files:
            file_dir_list.append(os.path.join(root, name))
    return file_dir_list, files
file_path = "result.csv"
write_result(['Index','File_name','CV_Procedure','n_rows','n_cols','EPE_DF','VAR_DF','Time_DF','EPE_RF','VAR_RF','Time_RF'],file_path)

file_names = ["auto.txt","analcatdata_asbestos.txt","analcatdata_boxing1.txt","iris.txt","postoperative-patient-data.txt"]
for i in range(5):
    file_name = file_names[i]
    path = os.path.join("randomforest_datasets",file_name)
    D = read_data(path)
    print("Deal with file:",file_name)

    print("---------------------BDS_Cross_Validation---------------------")
    m_df, v_df, t_df = BDS_Cross_Validation(D, "df", 5)
    print("Deep Forest:")
    result_out(m_df, v_df, t_df)
    m_rf, v_rf, t_rf = BDS_Cross_Validation(D, "rf", 5)
    print("Random Forest:")
    result_out(m_rf, v_rf, t_rf)
    write_result([i,file_name,'BDSCV',len(D),len(D[0]),m_df, v_df, t_df, m_rf, v_rf, t_rf],file_path)

    print("---------------------Monte_Carlo_Cross_Validation---------------------")
    m_df, v_df, t_df = Monte_Carlo_Cross_Validation(D, "df", 5, repeat=10)
    print("Deep Forest:")
    result_out(m_df, v_df, t_df)
    m_rf, v_rf, t_rf = Monte_Carlo_Cross_Validation(D, "rf", 5, repeat=10)
    print("Random Forest:")
    result_out( m_rf, v_rf, t_rf)
    write_result([i,file_name,'MCCV',len(D),len(D[0]),m_df, v_df, t_df, m_rf, v_rf, t_rf],file_path)
    print("Monte_Carlo_Cross_Validation_Stratified...")

    print("---------------------Monte_Carlo_Cross_Validation_Stratified---------------------")
    m_df, v_df, t_df = Monte_Carlo_Cross_Validation_Stratified(D, "df", 5)
    print("Deep Forest:")
    result_out(m_df, v_df, t_df)
    m_rf, v_rf, t_rf = Monte_Carlo_Cross_Validation_Stratified(D, "rf", 5)
    print("Random Forest:")
    result_out(m_rf, v_rf, t_rf)
    write_result([i,file_name,'Stratified_MCCV',len(D),len(D[0]),m_df, v_df, t_df, m_rf, v_rf, t_rf],file_path)

    print("---------------------LeaveOneOut---------------------")
    m_df, v_df, t_df = LeaveOneOut(D, "df")
    print("Deep Forest:")
    result_out(m_df, v_df, t_df)
    m_rf, v_rf, t_rf = LeaveOneOut(D, "rf")
    print("Random Forest:")
    result_out(m_rf, v_rf, t_rf)
    write_result([i,file_name,'LOO',len(D),len(D[0]),m_df, v_df, t_df, m_rf, v_rf, t_rf],file_path)


Deal with file: auto.txt
---------------------BDS_Cross_Validation---------------------
Deep Forest:
EPE:	0.8023170731707318
VAR:	0.00300898274836407
Time :	30.732688188552856s

Random Forest:
EPE:	0.797560975609756
VAR:	0.0020623140987507424
Time :	11.315667390823364s

---------------------Monte_Carlo_Cross_Validation---------------------
Deep Forest:
EPE:	0.7810365853658536
VAR:	0.005364910767400355
Time :	33.58285117149353s

Random Forest:
EPE:	0.7775121951219512
VAR:	0.005540523497917904
Time :	28.25880265235901s

Monte_Carlo_Cross_Validation_Stratified...
---------------------Monte_Carlo_Cross_Validation_Stratified---------------------
Deep Forest:
EPE:	0.7729268292682927
VAR:	0.004191046995835812
Time :	7.773919105529785s

Random Forest:
EPE:	0.7729268292682927
VAR:	0.004904907792980368
Time :	2.811765193939209s

---------------------LeaveOneOut---------------------
Deep Forest:
EPE:	0.8267326732673267
VAR:	0.14324576021958632
Time :	123.64608836174011s

Random Forest:
EPE:	0.826

## 4.7 结论
这边再次提醒读者注意查看final_result3.3.xlsx，所有的结果均存储在这个表格中。

我们直接给出结论，总体来说，在准确率和稳定上深度森林是**优于**随机森林的，在运行速度会上深度森林**劣于**随机森林。

因此，**深度森林是优于随机森林的。**

我们打开final_result.xlsx：

- 针对n_cols升序排序，我们可以得知，当数据集特征较少时，深度森林**劣于**随机森林。
- 针对n_cols和n_rows升序排序，我们可以得知，当数据集较小时，深度森林**劣于**随机森林。
- 针对n_cols降序排序，我们可以得知，当数据集特征较多时，深度森林**优于**随机森林。
- 针对n_cols和n_rows降序排序，我们可以得知，当数据集较大时，深度森林**优于**随机森林。

